Make sure to activate the `metaflow-structured-data` environment!

In [ ]:
import pyarrow.parquet as pq
import pyarrow
from metaflow import S3, profile
from concurrent.futures import ThreadPoolExecutor

In [ ]:
url = "s3://outerbounds-datasets/ubiquant/investment_ids"
N_FILES = 100

In [ ]:
stats = {}

with profile("read", stats_dict=stats):
    with S3() as s3:

        files = list(s3.list_recursive([url]))[:N_FILES]
        total_size = sum(f.size for f in files) / 1024**3
        print("Loading %2.1dGB of data" % total_size)
        stats = {}

        with profile('download', stats_dict=stats):
            loaded = s3.get_many([f.url for f in files])
        _print_throughput("S3->EC2 download", stats, total_size)

        with ThreadPoolExecutor(max_workers=num_threads) as exe:
            tables = exe.map(lambda f: pq.read_table(f, use_threads=False), files)
            table = pyarrow.concat_tables(tables)

# DuckDB

## Relational API

In [ ]:
import pyarrow.parquet as pq
import duckdb

COLUMN = "f_0"

# Reads Parquet File to an Arrow Table
arrow_table = pq.read_table('train_low_mem.parquet')

# Transforms Arrow Table -> DuckDB Relation
rel_from_arrow = duckdb.arrow(arrow_table)

# we can run a SQL query on this and print the result
res = rel_from_arrow.query('arrow_table', f'SELECT {COLUMN} FROM arrow_table;')

# Transforms DuckDB Relation -> Arrow Table
 arrow_table_from_duckdb = rel_from_arrow.arrow()

In [ ]:
res

## Query Directly with SQL + Replacement scans

In [ ]:
con = duckdb.connect()
res = con.execute(f'SELECT {COLUMN} FROM arrow_table;').fetch_arrow_table()

In [ ]:
res

# Resources

- [⏫ Fast Data Loading and Low Mem with Parquet Files](https://www.kaggle.com/code/robikscube/fast-data-loading-and-low-mem-with-parquet-files)
- https://gist.github.com/simicd/f0e8fcd277bb3fa932369551b97d5b07
- [DuckDB quacks Arrow: A zero-copy data integration between Apache Arrow and DuckDB](https://duckdb.org/2021/12/03/duck-arrow.html)